# Topic 16: AB Testing

* Power Analysis
* Run an A/B test

In [ ]:
import numpy as np
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt

# Power Analysis

As you've seen, power analysis allows you to determine the sample size required to detect an effect of a given size with a given degree of confidence. In other words, it allows you to determine the probability of detecting an effect of a given size with a given level of confidence, under-sample size constraints.

The following four factors have an intimate relationship:

1. Sample size ($n$)
2. Effect size ($d$)
3. Significance level ($\alpha$)= P (Type I error) = probability of finding an effect that is not there
4. Power ($\beta$) = 1 - P (Type II error) = probability of finding an effect that is there


Given any three of these, we can easily determine the fourth.

## Scenario

A researcher wants to study how daily protein supplementation in the elderly population will affect baseline liver fat. The study budget will allow enrollment of **24 patients**. Half will be randomized to a placebo group and half to the protein supplement treatment group and the trial will be carried out over one month. It is desired to see whether the mean change in percentage of liver fat from baseline to the end of the study differs between the two groups in the study.

### Setting Up Power Analysis Simulation

$$H_0: \mu_{1} = \mu_{2}$$
$$H_0: \mu_{1} \neq \mu_{2}$$

$$\alpha = 0.05$$

The researcher needs to know what power will be obtained under the sample size restrictions to identify a change in mean percent liver fat of **0.17**. Based on past results, a common standard deviation of 0.21 will be used for each treatment group in the power analysis.

In this experiment, we are constrained by our sample size (12), effect size (0.17 difference) and alpha (0.05). We want to find the **proportion of simulations where the null hypothesis is rejected**.



In [ ]:
# Number of patients in each group
sample_size = 12

# Control group
control_mean = 0
control_sd = 0.21

# Experimental group
experimental_mean = 0.17 # this is our mu2!!
experimental_sd = 0.21

# Set the number of simulations for our test = 1000
n_sim = 1000

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html

In [ ]:
# For reproducibility 
np.random.seed(10)

# Initialize array to store results
p = (np.empty(n_sim))
p.fill(np.nan)

#  Run a for loop for range of values in n_sim

for s in range(n_sim):
    
    # generating random variables with control/experimental parameters
    control = np.random.normal(loc= control_mean, scale=control_sd, size=sample_size)
    experimental = np.random.normal(loc= experimental_mean, scale=experimental_sd, size=sample_size)
    
    # running a 2-sample t test
    t_test = stats.ttest_ind(control, experimental)
    
    # t_test is a tuple containing: (t-statistic, p-value)
    p[s] = t_test[1]
    
print(p)

In [ ]:
# number of null hypothesis rejections
num_null_rejects = np.sum(p < 0.05)
power = num_null_rejects/float(n_sim)

power

## Achieving Power by Increasing Sample Size

Often in behavioral research 0.8 is accepted as a sufficient level of power.

https://www.statsmodels.org/stable/generated/statsmodels.stats.power.tt_ind_solve_power.html

In [ ]:
from statsmodels.stats.power import TTestIndPower
power = TTestIndPower()
power.solve_power(effect_size=0.17/0.21, alpha=0.05, power=0.8)

# round up for minimum sample size to achieve power = 0.8

### Iterative Approach

In [ ]:
# initializing
target = 0.8
sample_size = 12
null_rejected = 0
n_sim = 10000

np.random.seed(10)

p = (np.empty(n_sim))
p.fill(np.nan)

# Keep iterating until desired power is obtained

power_sample = []
while null_rejected < target:

    data = np.empty([n_sim, sample_size, 2])
    data.fill(np.nan)
    
    # For control group 
    # Here we specify size=[n_sim, sample_size] which creates an array of n_sim number of arrays,
    # each containing sample_size number of elements. 
    # This is equivalent to manually looping n_sim times like we did above but is much faster.
    data[:,:,0] = np.random.normal(loc=control_mean, scale=control_sd, size=[n_sim, sample_size])
    
    # For experimental group
    data[:,:,1] = np.random.normal(loc=experimental_mean, scale=experimental_sd, size=[n_sim, sample_size])            
    
    result = stats.ttest_ind(data[:, :, 0],data[:, :, 1],axis=1)
                                
    p_vals = result[1]

    # Since you know that all simulations are from a different distribution \
    # all those that rejected the null-hypothesis are valid
    null_rejected = np.sum(p_vals < 0.05) / n_sim

    print('Number of Samples:', sample_size,', Calculated Power =', null_rejected)
    power_sample.append([sample_size, null_rejected])

    # increase the number of samples by one for the next iteration of the loop
    sample_size += 1

In [ ]:
plt.figure(figsize=(10,5))
plt.title('Power vs. Sample Size')
plt.xlabel('Sample Size')
plt.ylabel('Power')

ans = power_sample
df = pd.DataFrame(ans, index=None)
plt.plot(df[0], df[1])

plt.show()

# AB Testing

Let's see if a change to a webpage led to more clicks.

In [ ]:
df = pd.read_csv('homepage_actions.csv')
df

### A Quick EDA...

In [ ]:
cids = set(df[df.action=='click']['id'].unique())
vids = set(df[df.action=='view']['id'].unique())
print("Number of viewers: {} \tNumber of clickers: {}".format(len(vids), len(cids)))
print("Number of viewers who didn't click: {}".format(len(vids-cids)))
print("Number of clickers who didn't view: {}".format(len(cids-vids)))

In [ ]:
control = df[df['group']=='control']
experiment = df[df['group']=='experiment']

In [ ]:
df['count'] = 1
df.head()

In [ ]:
#Convert clicks into a binary variable on a user-by-user-basis
control = df[df.group=='control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)

experiment = df[df.group=='experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)

print("Sample sizes:\tControl: {}\tExperiment: {}".format(len(control), len(experiment)))
print("Total Clicks:\tControl: {}\tExperiment: {}".format(control.click.sum(), experiment.click.sum()))
print("Average click rate:\tControl: {}\tExperiment: {}".format(control.click.mean(), experiment.click.mean()))
control.head(10)

### t-test formulas from old labs

In [ ]:
def welch_t(a, b):
    
    """ Calculate Welch's t statistic for two samples. """

    numerator = a.mean() - b.mean()
    
    # “ddof = Delta Degrees of Freedom”: the divisor used in the calculation is N - ddof, 
    #  where N represents the number of elements. By default ddof is zero.
    
    denominator = np.sqrt(a.var(ddof=1)/a.size + b.var(ddof=1)/b.size)
    
    return np.abs(numerator/denominator)

def welch_df(a, b):
    
    """ Calculate the effective degrees of freedom for two samples. This function returns the degrees of freedom """
    
    s1 = a.var(ddof=1) 
    s2 = b.var(ddof=1)
    n1 = a.size
    n2 = b.size
    
    numerator = (s1/n1 + s2/n2)**2
    denominator = (s1/ n1)**2/(n1 - 1) + (s2/ n2)**2/(n2 - 1)
    
    return numerator/denominator


def p_value_welch_ttest(a, b, two_sided=False):
    """Calculates the p-value for Welch's t-test given two samples.
    By default, the returned p-value is for a one-sided t-test. 
    Set the two-sided parameter to True if you wish to perform a two-sided t-test instead.
    """
    t = welch_t(a, b)
    df = welch_df(a, b)
    
    p = 1-stats.t.cdf(np.abs(t), df)
    
    if two_sided:
        return 2*p
    else:
        return p


In [ ]:
p_value_welch_ttest(control.click, experiment.click, two_sided=True)


In [ ]:
# or using scipy:

stats.ttest_ind(control.click, experiment.click)